# Friend or Foe: Challenge & Response System (CRS)

## Objective 


Although the camouflage identification system will provide initial friend/foe identification, an AI model alone is not sufficient when exposed to an adversarial actor. 

## Physical Architecture

The challenge & response system has 2 physical parts:
1. Enhanced Weapon System (EWS)

    * Weapon optic system with AI model
    * Long range directional antenna
    * Software defined radio (SDR)

2. Soldier Uniforms with Embedded Receiver / Transponder Units

### Enhanced Weapon System (EWS) Architecture

![Weapon Image](./ar-image.png "EWS Image")

### Software Architecture

From a software perspective, there are 2 components:
    
1. Challenge request & validation system embedded within the SDR. For our purposes can be referred to as the server. The server is embedded into the firmware of the SDR.

2. Response system embedded into the firmware of the military uniforms. This portion is the client.

## Running Code

Although you are welcome to view the python code here, you must run it in outside the scope of Jupiter to successfully listen for connections as the server, and make connections as the client.

## Server Code (Run in Terminal)

In [1]:
import socket

targetChallenge = "<missionSpecificPassword>"
lengthOfMission = 1 # days until timeout

def serverMain():
    hostname = socket.gethostname()
    port = 9999 
    server_socket = socket.socket()  # get instance
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # allow the socket to be reused
    server_socket.settimeout(86400 * lengthOfMission) # Only wait 3 seconds for data, otherwise determine no data is provided
    server_socket.bind((hostname, port))  # bind host address and port together
    server_socket.listen(1)  # configure how many clients the server can listen simultaneously (1 target at a time for our use)
    print("Server is online & awaiting connections. Point the firearm at a target.")
    
    try:
        while True:
            conn, address = server_socket.accept()
            if(conn, address):
                print("Connection from: " + str(address))
                
                # receive data stream. it won't accept data packet greater than 1024 bytes
                data = conn.recv(1024).decode()
                if data:
                    if targetChallenge in data:
                        targetIsFriendly(address)
                    else:
                        targetIsUnknown(address)
                        print(f"Data received:{data}")
                        # Why is target unknown if they fail the challenge/response test?
                        # A target may fail the challenge response check for several reasons, not all of which indicate a foe.
                        # For example, if a friendly target has become detached from his group and during which time, the passcode has changed, they are friendly and should not be killed, however they will fail CRS.
                else:
                    break
    except socket.timeout:
            onMissionTimeout()
    finally:
        try:
            conn.close()  # always close the connection
            print("Connection closed")
        except:
            pass
    
def targetIsFriendly(address):
    print(f"Target at address: {address} is a friendly. DO NOT SHOOT!")
    # Update optic system to represent friendly target
    
def targetIsUnknown(address):
    print(f"Target at address: {address} is a unknown. Proceed with caution")
    # Update optic system to represent unknown target
    
def onMissionTimeout():
    print("System has timed-out.")
    exit(1)
    
serverMain()

Server is online & awaiting connections. Point the firearm at a target.
System has timed-out.


## Friendly Client Code (Run in Terminal) -- Server Will Identify as Friendly

In [2]:
import socket

targetResponse = "<missionSpecificPassword>"

def clientMain():
    print("Client started")
    hostname = socket.gethostname()  # as both code is running on same computer
    port = 9999  # socket server port number
    client_socket = socket.socket()  # instantiate
    client_socket.connect((hostname, port))  # connect to the server
    print("Sending response to server...")
    client_socket.send(targetResponse.encode())  # send message
    print("Response sent")
    client_socket.close()  # close the connection

clientMain()

Client started
Sending response to server...
Response sent


## Unknown Client Code (Run in Terminal) -- Server Will Identify as Unknown

In [ ]:
import socket

targetResponse = "someOtherPassword"

def clientMain():
    print("Client started")
    hostname = socket.gethostname()  # as both code is running on same computer
    port = 9999  # socket server port number
    client_socket = socket.socket()  # instantiate
    client_socket.connect((hostname, port))  # connect to the server
    print("Sending response to server...")
    client_socket.send(targetResponse.encode())  # send message
    print("Response sent")
    client_socket.close()  # close the connection

clientMain()

# Important!!

**Due to the TCP protocol being overly concerned with syn/acking all potential packets, despite the fact that we closed the connection of the server, it will remain active for a few mins, sitting in the TIME_WAIT TCP State and waiting for a final packet.This is a security risk!**

For more information on why this is, please see: https://en.wikipedia.org/wiki/Transmission_Control_Protocol

Please be sure to run the code below <u>in your terminal, not in this notebook</u> to actually kill the socket.

In [ ]:
# Run this command in your terminal to fully close the socket.
kill $(lsof -t -i:9999)

## Verify that the socket is truly free

In [ ]:
# Run this command in your terminal, if you get results it is not free, if you don't it is:
lsof -t -i:9999

### Considerations

The goal of the CRS is merely to confirm the classification of the AI model, not to stand-alone. Without the aid of the AI system, the overall friend-or-foe system is not suitable for operational use. The CRS has been designed to be fail-safe, meaning that if an error occurs or if the system is not sure wether a target should be confirmed as friendly, it will not be. Furthermore, the system has been purpose built to render itself inoperable should the system fall into the hands of an enemy, further ensuring that the system is not turned against the operator.

## Sources:

\[1]: https://www.kindpng.com/picc/m/250-2508509_transparent-ar15-clipart-outline-ar-15-drawing-hd.png

\[2]: https://rfengineer.net/wp-content/uploads/2021/03/What-do-I-need-for-SDR-radio-1.png

\[3]: https://www.digitalocean.com/community/tutorials/python-socket-programming-server-client